In [15]:
import requests
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime

# Загрузка настроек из файла config.json
with open('config.json') as config_file:
    config = json.load(config_file)

# Аутентификация в Google Sheets API
scope = ['https://www.googleapis.com/auth/spreadsheets']
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
gc = gspread.authorize(credentials)

# Функция для обновления или создания листа в Google Sheets с заголовками столбцов
def update_or_create_sheet(spreadsheet, sheet_title, rows):
    try:
        sheet = spreadsheet.worksheet(sheet_title)
        sheet.clear()
        sheet.resize(rows=len(rows) + 1)
        sheet.insert_rows(rows)
    except gspread.WorksheetNotFound:
        sheet = spreadsheet.add_worksheet(title=sheet_title, rows=len(rows) + 1, cols=11)
        sheet.insert_rows(rows)

    headers = [
        "Дата и время проверки",  # Дата и время проверки
        "Дата публикации",  # Дата публикации
        "Ссылка на пост",  # Ссылка на пост
        "Текст",  # Текст
        "Комментарии",  # Комментарии
        "Лайки",  # Лайки
        "Репосты",  # Репосты
        "Просмотры",  # Просмотры
        "Post ID", "Hide", "Join Group", "Links", "Reach Subscribers",
        "Reach Total", "Reach Viral", "Reach Ads", "Report", "To Group", "Unsubscribe"
    ]
    sheet.insert_row(headers, 1)

url = 'https://api.vk.com/method/stats.getPostReach'
params = {
    'access_token': config['token_My'],
    'v': '5.131',
    'extended': 0,
    'filter': 'reach',
}

groups = config['groups']

for group_id in groups:
    print(f"Статистика постов для сообщества с ID {group_id}:")

    wall_get_params = {
        'access_token': config['token_My'],
        'v': '5.131',
        'owner_id': group_id,
        'count': 100,
        'offset': 0,
        'filter': 'all',
    }

    response = requests.get('https://api.vk.com/method/wall.get', params=wall_get_params)
    data = response.json()

    if 'response' in data and 'items' in data['response']:
        posts = data['response']['items']
        post_count = len(posts)
        batch_count = (post_count // 30) + 1
        stats = []

        for i in range(batch_count):
            batch_start = i * 30
            batch_end = (i + 1) * 30
            wall_posts_batch = posts[batch_start:batch_end]
            post_ids_batch = [str(post['id']) for post in wall_posts_batch]

            params['owner_id'] = group_id
            params['post_ids'] = ','.join(post_ids_batch)

            response = requests.get(url, params=params)
            data = response.json()

            if 'response' in data:
                post_reach_list = data['response']

                for post, post_reach in zip(wall_posts_batch, post_reach_list):
                    post_stats = [
                        datetime.now().strftime("%Y-%m-%d %H:%M:%S"),  # Дата и время проверки
                        post['date'],  # Дата публикации
                        f"https://vk.com/wall{post['owner_id']}_{post['id']}",  # Ссылка на пост
                        post['text'],  # Текст
                        post['comments']['count'],  # Комментарии
                        post['likes']['count'],  # Лайки
                        post['reposts']['count'],  # Репосты
                        post.get('views', {}).get('count', 0),  # Просмотры
                        post_reach['post_id'], post_reach.get('hide', 0), post_reach.get('join_group', 0),
                        post_reach.get('links', 0), post_reach.get('reach_subscribers', 0),
                        post_reach.get('reach_total', 0), post_reach.get('reach_viral', 0),
                        post_reach.get('reach_ads', 0), post_reach.get('report', 0),
                        post_reach.get('to_group', 0), post_reach.get('unsubscribe', 0)
                    ]
                    stats.append(post_stats)
            else:
                print(f"Ошибка при получении статистики для постов сообщества {group_id}: {data.get('error', 'Неизвестная ошибка')}")

        target_spreadsheet_id = config['target_spreadsheet_id']
        target_spreadsheet = gc.open_by_key(target_spreadsheet_id)
        sheet_title = str(group_id)
        update_or_create_sheet(target_spreadsheet, sheet_title, stats)

    else:
        print(f"Ошибка при получении списка постов для сообщества {group_id}: {data.get('error', 'Неизвестная ошибка')}")


Статистика постов для сообщества с ID -6649905:
Статистика постов для сообщества с ID -1623346:
